In [9]:
import os
import pandas as pd
from ragas.metrics import answer_relevancy, answer_correctness
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain.chat_models import ChatOpenAI
from langchain_localai import LocalAIEmbeddings
from datasets import Dataset
from dotenv import load_dotenv
import json

In [10]:
load_dotenv()

INPUT_FILE = os.getenv("INPUT_FILE")
OUTPUT_FILE = os.getenv("OUTPUT_FILE")

In [11]:
print("\n\n ################ RAG METRIC EVALUATION ###################")
print(f"Input file: {INPUT_FILE}")
print(f"Output file: {OUTPUT_FILE}")
print("###########################################################\n\n")



 ################ RAG METRIC EVALUATION ###################
Input file: ./test_data/study_rules/test_set_method1_lightrag.json
Output file: ./results/study_rules/method1_lighrag/score.csv
###########################################################




In [12]:
with open(INPUT_FILE, 'r', encoding='utf-8') as file:
    data = json.load(file)
dataset = Dataset.from_dict(data)

In [13]:
llm = ChatOpenAI(
    openai_api_base=os.getenv("OPENAI_API_BASE"), 
    openai_api_key=os.getenv("OPENAI_API_KEY"),              
    model_name=os.getenv("LLM_MODEL_NAME")          
)
llm = LangchainLLMWrapper(llm)

embedding = LocalAIEmbeddings(
    openai_api_base=os.getenv("EMBED_URL"), 
    openai_api_key=os.getenv("EMBED_TOKEN"), 
    model=os.getenv("EMBED_MODEL")
)
embedding = LangchainEmbeddingsWrapper(embedding)

results = evaluate(
    dataset=dataset,
    metrics=[answer_correctness, answer_relevancy],
    llm=llm,
    embeddings=embedding
)

df = results.to_pandas()
df.to_csv(OUTPUT_FILE,index=False)
print(f"Done - saved in: {OUTPUT_FILE}")

/tmp/ipykernel_8094/1435345648.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
Evaluating: 100%|██████████| 18/18 [00:56<00:00,  3.15s/it]


Done - saved in: ./results/study_rules/method1_lighrag/score.csv


In [14]:
df = pd.read_csv(OUTPUT_FILE)
df

,user_input,response,reference,answer_correctness,answer_relevancy
0,What is the procedure and timeline for a stude...,**Procedure for Transfer from Another Universi...,A student may transfer from another university...,0.766353,0.928777
1,What types of training are students required t...,**Training Requirements for First Semester Stu...,Students are required to complete training in ...,0.739027,0.931348
2,How does Gdańsk University of Technology defin...,Confirming learning outcomes for admission at ...,Confirmation of learning outcomes is conducted...,0.721505,0.923147
3,What are the responsibilities of the Faculty S...,The Faculty Student Council is a representativ...,The Faculty Student Council is responsible for...,0.552259,0.772437
4,Under what circumstances can classes from the ...,"Based on the provided Knowledge Base, I was un...",Such classes may be conducted in a foreign lan...,0.141376,0.000000
5,What procedures must academic teachers follow ...,"According to the Knowledge Base, academic teac...",Academic teachers complete subject cards in th...,0.423226,0.796308
6,What are the responsibilities of academic teac...,"According to the Knowledge Base, academic teac...",Academic teachers must register student attend...,0.236015,0.531554
7,Under what circumstances can a student be remo...,**Removal from Classes**\n\nAccording to the p...,A student can be removed from classes if their...,0.268342,0.935541
8,"How is the weighted average grade calculated, ...",**Calculation of Weighted Average Grade**\n\nA...,The weighted average grade is calculated by di...,0.181425,0.995617
